# Coreference Resolution con XLM-RoBERTa
## Modelo para identificar clusters de coreferencia

## 1. Instalación de Dependencias

In [ ]:
# Instalación de paquetes necesarios
# !pip install -q transformers torch datasets numpy scikit-learn spacy matplotlib tqdm wandb
# !python -m spacy download es_core_news_sm
# !python -m spacy download en_core_web_sm

## 2. Imports y Configuración

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
import random
import os
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from collections import defaultdict

# Transformers y NLP
from transformers import (
    XLMRobertaModel, 
    XLMRobertaTokenizer, 
    XLMRobertaConfig,
    AdamW,
    get_linear_schedule_with_warmup
)

# Datos y visualización
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import spacy

# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo: {device}")

# Configuración de reproducción
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

## 3. Definición del Modelo de Coreferencia

In [ ]:
class SpanRepresentation(nn.Module):
    """Módulo para representar spans de texto"""
    
    def __init__(self, hidden_size: int, max_span_width: int = 10):
        super().__init__()
        self.hidden_size = hidden_size
        self.max_span_width = max_span_width
        
        # Capas para embeddings de inicio y fin
        self.start_mlp = nn.Linear(hidden_size, hidden_size)
        self.end_mlp = nn.Linear(hidden_size, hidden_size)
        
        # Embeddings para el ancho del span
        self.span_width_embeddings = nn.Embedding(max_span_width, hidden_size)
        
        # Atención para tokens internos del span
        self.span_attention = nn.Linear(hidden_size, 1)
        
        # Capa de normalización
        self.layer_norm = nn.LayerNorm(hidden_size)
        
    def forward(self, sequence_output: torch.Tensor, 
                span_indices: List[Tuple[int, int]]) -> torch.Tensor:
        """
        Args:
            sequence_output: [batch_size, seq_len, hidden_size]
            span_indices: Lista de (start_idx, end_idx) por batch
        Returns:
            span_embeddings: [batch_size, num_spans, hidden_size]
        """
        batch_size, seq_len, hidden_size = sequence_output.shape
        num_spans = len(span_indices[0])  # Asumimos mismo número de spans por batch
        
        span_embeddings = []
        
        for b in range(batch_size):
            batch_spans = []
            
            for start_idx, end_idx in span_indices[b]:
                # Asegurar que el span sea válido
                if start_idx >= seq_len or end_idx >= seq_len or start_idx > end_idx:
                    # Span inválido, usar vector cero
                    batch_spans.append(torch.zeros(hidden_size, device=device))
                    continue
                
                # 1. Embeddings de inicio y fin
                start_emb = sequence_output[b, start_idx, :]
                end_emb = sequence_output[b, end_idx, :]
                
                start_proj = self.start_mlp(start_emb)
                end_proj = self.end_mlp(end_emb)
                
                # 2. Embedding del ancho del span
                span_width = min(end_idx - start_idx, self.max_span_width - 1)
                width_idx = torch.tensor(span_width, device=device)
                width_emb = self.span_width_embeddings(width_idx)
                
                # 3. Atención sobre los tokens internos
                if end_idx > start_idx:
                    span_tokens = sequence_output[b, start_idx:end_idx+1, :]
                    attention_weights = F.softmax(
                        self.span_attention(span_tokens), dim=0
                    )
                    attended_rep = torch.sum(attention_weights * span_tokens, dim=0)
                else:
                    attended_rep = sequence_output[b, start_idx, :]
                
                # 4. Combinar representaciones
                span_rep = start_proj + end_proj + width_emb + attended_rep
                span_rep = self.layer_norm(span_rep)
                
                batch_spans.append(span_rep)
            
            span_embeddings.append(torch.stack(batch_spans))
        
        return torch.stack(span_embeddings)  # [batch_size, num_spans, hidden_size]

class CoreferenceScorer(nn.Module):
    """Calcula scores de coreferencia entre pares de spans"""
    
    def __init__(self, hidden_size: int, feature_size: int = 128):
        super().__init__()
        
        # Features para distancia y otros metadatos
        self.distance_embeddings = nn.Embedding(50, 20)  # Distancias hasta 50 tokens
        self.same_sentence_emb = nn.Embedding(2, 10)     # ¿Misma oración?
        self.span_type_emb = nn.Embedding(3, 10)         # Tipo de span
        
        # Capas para combinar representaciones
        self.span_pair_mlp = nn.Sequential(
            nn.Linear(hidden_size * 3 + 40, 512),  # 40 de features adicionales
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1)
        )
        
    def forward(self, span_embeddings: torch.Tensor, 
                span_pairs: List[List[Tuple[int, int]]],
                metadata: Optional[Dict] = None) -> torch.Tensor:
        """
        Args:
            span_embeddings: [batch_size, num_spans, hidden_size]
            span_pairs: Lista de pares (i, j) por batch
            metadata: Diccionario con metadatos adicionales
        Returns:
            scores: [batch_size, num_pairs]
        """
        batch_size, num_spans, hidden_size = span_embeddings.shape
        scores = []
        
        for b in range(batch_size):
            batch_scores = []
            
            for i, j in span_pairs[b]:
                if i >= num_spans or j >= num_spans:
                    batch_scores.append(torch.tensor(-1e10, device=device))
                    continue
                
                # Representaciones de los spans
                span_i = span_embeddings[b, i, :]
                span_j = span_embeddings[b, j, :]
                
                # Features del par
                distance = min(abs(i - j), 49)
                distance_feat = self.distance_embeddings(
                    torch.tensor(distance, device=device)
                )
                
                # ¿Misma oración? (simplificado)
                same_sent = 1 if abs(i - j) < 20 else 0  # Heurística simple
                same_sent_feat = self.same_sentence_emb(
                    torch.tensor(same_sent, device=device)
                )
                
                # Producto punto entre embeddings
                interaction = span_i * span_j
                
                # Concatenar todo
                pair_features = torch.cat([
                    span_i,
                    span_j,
                    interaction,
                    distance_feat,
                    same_sent_feat
                ])
                
                # Calcular score
                score = self.span_pair_mlp(pair_features.unsqueeze(0))
                batch_scores.append(score.squeeze())
            
            scores.append(torch.stack(batch_scores) if batch_scores else torch.tensor([], device=device))
        
        return scores

class CoreferenceClusterModel(nn.Module):
    """Modelo principal para resolución de coreferencias"""
    
    def __init__(self, 
                 model_name: str = "xlm-roberta-base",
                 max_span_width: int = 10,
                 max_num_spans: int = 100):
        super().__init__()
        
        # Modelo base XLM-RoBERTa
        self.xlmr = XLMRobertaModel.from_pretrained(model_name)
        self.hidden_size = self.xlmr.config.hidden_size
        
        # Componentes del modelo
        self.span_representation = SpanRepresentation(
            hidden_size=self.hidden_size,
            max_span_width=max_span_width
        )
        
        self.coreference_scorer = CoreferenceScorer(
            hidden_size=self.hidden_size
        )
        
        # Clasificador para dummy antecedent
        self.dummy_antecedent = nn.Parameter(torch.randn(1, self.hidden_size))
        self.dummy_scorer = nn.Linear(self.hidden_size, 1)
        
        # Configuraciones
        self.max_span_width = max_span_width
        self.max_num_spans = max_num_spans
        
        # Inicialización
        self.init_weights()
        
    def init_weights(self):
        """Inicialización de pesos"""
        nn.init.xavier_uniform_(self.dummy_antecedent)
        
    def extract_candidate_spans(self, 
                               sequence_output: torch.Tensor,
                               attention_mask: torch.Tensor) -> List[List[Tuple[int, int]]]:
        """
        Extrae spans candidatos del texto
        
        Args:
            sequence_output: [batch_size, seq_len, hidden_size]
            attention_mask: [batch_size, seq_len]
        Returns:
            span_indices: Lista de listas de (start, end)
        """
        batch_size, seq_len, _ = sequence_output.shape
        span_indices = []
        
        for b in range(batch_size):
            # Encontrar tokens reales (no padding)
            real_tokens = torch.where(attention_mask[b] == 1)[0]
            if len(real_tokens) == 0:
                span_indices.append([])
                continue
                
            last_token = real_tokens[-1].item()
            batch_spans = []
            
            # Generar spans de todos los anchos posibles
            for start in range(last_token + 1):
                for width in range(self.max_span_width):
                    end = start + width
                    if end > last_token:
                        break
                    batch_spans.append((start, end))
            
            # Limitar número de spans
            if len(batch_spans) > self.max_num_spans:
                # Priorizar spans más cortos
                batch_spans = sorted(batch_spans, 
                                   key=lambda x: (x[1] - x[0], x[0]))[:self.max_num_spans]
            
            span_indices.append(batch_spans)
        
        return span_indices
    
    def create_span_pairs(self, 
                         span_indices: List[List[Tuple[int, int]]]) -> List[List[Tuple[int, int]]]:
        """
        Crea todos los pares posibles entre spans (i, j) donde j es anterior a i
        
        Args:
            span_indices: Lista de spans por batch
        Returns:
            span_pairs: Lista de pares (i, j) por batch
        """
        span_pairs = []
        
        for batch_spans in span_indices:
            num_spans = len(batch_spans)
            batch_pairs = []
            
            for i in range(num_spans):
                for j in range(i):  # Solo spans anteriores
                    batch_pairs.append((i, j))
            
            span_pairs.append(batch_pairs)
        
        return span_pairs
    
    def forward(self, 
                input_ids: torch.Tensor,
                attention_mask: torch.Tensor,
                return_spans: bool = False):
        """
        Forward pass del modelo
        
        Args:
            input_ids: [batch_size, seq_len]
            attention_mask: [batch_size, seq_len]
        Returns:
            Dict con scores y spans
        """
        # 1. Obtener embeddings contextuales
        outputs = self.xlmr(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        sequence_output = outputs.last_hidden_state
        
        # 2. Extraer spans candidatos
        span_indices = self.extract_candidate_spans(sequence_output, attention_mask)
        
        # 3. Obtener representaciones de spans
        span_embeddings = self.span_representation(sequence_output, span_indices)
        
        # 4. Crear pares de spans
        span_pairs = self.create_span_pairs(span_indices)
        
        # 5. Calcular scores de coreferencia
        pairwise_scores = self.coreference_scorer(span_embeddings, span_pairs)
        
        # 6. Añadir dummy antecedent scores
        final_scores = []
        for b in range(len(pairwise_scores)):
            if len(pairwise_scores[b]) == 0:
                final_scores.append(torch.tensor([], device=device))
                continue
            
            # Reorganizar scores por span
            num_spans = len(span_indices[b])
            span_scores = []
            
            # Para cada span i, tenemos scores con spans anteriores j
            pair_idx = 0
            for i in range(num_spans):
                antecedent_scores = []
                
                # Scores con spans anteriores
                for j in range(i):
                    antecedent_scores.append(pairwise_scores[b][pair_idx])
                    pair_idx += 1
                
                # Score con dummy antecedent (ninguno)
                dummy_score = self.dummy_scorer(self.dummy_antecedent).squeeze()
                antecedent_scores.append(dummy_score)
                
                span_scores.append(torch.stack(antecedent_scores))
            
            final_scores.append(torch.stack(span_scores) if span_scores else torch.tensor([], device=device))
        
        if return_spans:
            return {
                'scores': final_scores,
                'span_indices': span_indices,
                'span_embeddings': span_embeddings
            }
        
        return final_scores

- Limitaciones del diseño actual:

Problema: Número exponencial de spans

num_spans ≈ O(n * max_span_width)  # n = número de tokens

Solución práctica: max_span_width = 10, max_num_spans = 100

- Optimizaciones posibles:

En lugar de todos los spans, usar heurísticas:

Solo sustantivos y pronombres (usando POS tags)

Solo menciones con cabeza nominal (dependency parsing)

Filtrar spans muy largos (>5 tokens rara vez son menciones)

## 4. Dataset y Preprocesamiento

In [ ]:
import conllu
import re
from typing import List, Dict, Tuple, Optional

@dataclass
class CoreferenceExample:
    """Estructura para un ejemplo de coreferencia"""
    text: str
    tokens: List[str]
    clusters: List[List[Tuple[int, int]]]  # [[(start1, end1), (start2, end2)], ...]
    char_clusters: List[List[Tuple[int, int]]]  # Clusters en offsets de caracteres
    
    @classmethod
    def from_dict(cls, data: Dict):
        """Crea un ejemplo desde un diccionario"""
        return cls(
            text=data['text'],
            tokens=data.get('tokens', []),
            clusters=data['clusters'],
            char_clusters=data.get('char_clusters', [])
        )

class CoNLLUReader:
    """Lector para archivos CoNLL-U con anotaciones de coreferencia"""
    
    @staticmethod
    def parse_coref_field(coref_str: str) -> List[Tuple[int, str]]:
        """
        Parsea el campo de coreferencia en formato CoNLL-U
        
        Formatos:
        - (123        → inicio del cluster 123
        - 123)        → fin del cluster 123
        - (123)       → cluster 123 de un solo token
        - 123         → continuación del cluster 123
        - (123|(456   → múltiples clusters
        """
        if coref_str == '_':
            return []
        
        annotations = []
        # Separar múltiples anotaciones (ej: "(1|(5")
        parts = coref_str.split('|')
        
        for part in parts:
            # Expresión regular para capturar tags de coreferencia
            if re.match(r'^\(\d+\)$', part):  # (123)
                cluster_id = int(part[1:-1])
                annotations.append((cluster_id, 'single'))
            elif re.match(r'^\(\d+$', part):  # (123
                cluster_id = int(part[1:])
                annotations.append((cluster_id, 'start'))
            elif re.match(r'^\d+\)$', part):  # 123)
                cluster_id = int(part[:-1])
                annotations.append((cluster_id, 'end'))
            elif re.match(r'^\d+$', part):  # 123
                cluster_id = int(part)
                annotations.append((cluster_id, 'middle'))
        
        return annotations
    
    @classmethod
    def load_from_conllu(cls, filepath: str) -> List[CoreferenceExample]:
        """
        Carga un archivo CoNLL-U y lo convierte en CoreferenceExample
        
        Args:
            filepath: Ruta al archivo .conllu
        
        Returns:
            Lista de CoreferenceExample
        """
        examples = []
        
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Dividir por documentos (separados por líneas vacías o # newdoc)
        doc_blocks = []
        current_block = []
        
        for line in content.split('\n'):
            if line.strip() == '' or line.startswith('# newdoc'):
                if current_block:
                    doc_blocks.append(current_block)
                    current_block = []
                if line.startswith('# newdoc'):
                    current_block.append(line)
            else:
                current_block.append(line)
        
        if current_block:
            doc_blocks.append(current_block)
        
        # Procesar cada documento
        for doc_lines in doc_blocks:
            example = cls.parse_conllu_document(doc_lines)
            if example:
                examples.append(example)
        
        print(f"✅ Cargados {len(examples)} ejemplos desde {filepath}")
        return examples
    
    @classmethod
    def parse_conllu_document(cls, lines: List[str]) -> Optional[CoreferenceExample]:
        """
        Parsea un documento CoNLL-U individual
        
        Returns:
            CoreferenceExample o None si no hay clusters
        """
        # Filtrar comentarios y líneas vacías
        token_lines = [line for line in lines if line.strip() and not line.startswith('#')]
        
        if not token_lines:
            return None
        
        # Reconstruir texto y tokens
        tokens = []
        char_offset = 0
        text_parts = []
        coref_annotations = []  # (token_idx, cluster_id, position_type)
        
        for line in token_lines:
            parts = line.split('\t')
            if len(parts) < 10:  # CoNLL-U básico tiene 10 columnas
                continue
            
            token_id = parts[0]
            token_form = parts[1]
            coref_field = parts[9] if len(parts) > 9 else '_'  # Última columna para coref
            
            # Añadir token
            tokens.append(token_form)
            
            # Añadir al texto reconstruido
            if text_parts:
                text_parts.append(' ')
                char_offset += 1
            
            text_parts.append(token_form)
            token_char_start = char_offset
            token_char_end = char_offset + len(token_form)
            
            # Procesar anotaciones de coreferencia
            if coref_field != '_':
                annotations = cls.parse_coref_field(coref_field)
                for cluster_id, pos_type in annotations:
                    coref_annotations.append({
                        'token_idx': len(tokens) - 1,
                        'cluster_id': cluster_id,
                        'position_type': pos_type,
                        'char_start': token_char_start,
                        'char_end': token_char_end
                    })
            
            char_offset = token_char_end
        
        # Reconstruir texto completo
        text = ''.join(text_parts)
        
        # Construir clusters desde anotaciones token-level
        clusters, char_clusters = cls.build_clusters_from_annotations(
            coref_annotations, tokens, text
        )
        
        # Solo devolver si hay clusters
        if not clusters:
            return None
        
        return CoreferenceExample(
            text=text,
            tokens=tokens,
            clusters=clusters,
            char_clusters=char_clusters
        )
    
    @staticmethod
    def build_clusters_from_annotations(annotations: List[Dict], 
                                      tokens: List[str],
                                      text: str) -> Tuple[List, List]:
        """
        Construye clusters a partir de anotaciones token-level
        
        Returns:
            (token_clusters, char_clusters)
        """
        # Agrupar por cluster_id
        clusters_by_id = {}
        for ann in annotations:
            cluster_id = ann['cluster_id']
            if cluster_id not in clusters_by_id:
                clusters_by_id[cluster_id] = []
            clusters_by_id[cluster_id].append(ann)
        
        # Construir spans para cada cluster
        token_clusters = []
        char_clusters = []
        
        for cluster_id, ann_list in clusters_by_id.items():
            # Ordenar por token_idx
            ann_list.sort(key=lambda x: x['token_idx'])
            
            spans = []
            char_spans = []
            current_span = None
            current_char_span = None
            
            i = 0
            while i < len(ann_list):
                ann = ann_list[i]
                pos_type = ann['position_type']
                
                if pos_type == 'single':
                    # Mención de un solo token
                    spans.append([ann['token_idx'], ann['token_idx']])
                    char_spans.append([ann['char_start'], ann['char_end']])
                    i += 1
                
                elif pos_type == 'start':
                    # Inicio de span multi-token
                    current_span = [ann['token_idx']]
                    current_char_span = [ann['char_start']]
                    i += 1
                    
                    # Buscar el fin
                    while i < len(ann_list) and ann_list[i]['position_type'] != 'end':
                        i += 1
                    
                    if i < len(ann_list) and ann_list[i]['position_type'] == 'end':
                        # Añadir fin
                        current_span.append(ann_list[i]['token_idx'])
                        current_char_span.append(ann_list[i]['char_end'])
                        spans.append(current_span)
                        char_spans.append(current_char_span)
                        i += 1
                    else:
                        # Span sin fin - tratar como single
                        spans.append([ann['token_idx'], ann['token_idx']])
                        char_spans.append([ann['char_start'], ann['char_end']])
                
                else:
                    i += 1  # Saltar 'middle' o 'end' sin inicio
            
            # Solo añadir clusters con al menos 2 menciones
            if len(spans) >= 2:
                token_clusters.append(spans)
                char_clusters.append(char_spans)
        
        return token_clusters, char_clusters

class CoreferenceDataset(Dataset):
    """Dataset para entrenamiento de coreferencia - Versión mejorada para CoNLL-U"""
    
    def __init__(self, 
                 examples: List[CoreferenceExample],
                 tokenizer: XLMRobertaTokenizer,
                 max_length: int = 512,
                 max_spans: int = 100,
                 is_training: bool = True):
        
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_spans = max_spans
        self.is_training = is_training
        
    def __len__(self):
        return len(self.examples)
    
    def tokenize_and_align(self, text: str):
        """Tokeniza el texto y obtiene mapping de caracteres a tokens"""
        encoding = self.tokenizer(
            text,
            return_offsets_mapping=True,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length
        )
        
        return encoding
    
    def align_clusters_to_tokens(self, 
                                offset_mapping: List[Tuple[int, int]],
                                char_clusters: List[List[Tuple[int, int]]]):
        """
        Convierte clusters de caracteres a clusters de tokens
        
        Args:
            offset_mapping: Lista de (start_char, end_char) por token
            char_clusters: Clusters en offsets de caracteres
        Returns:
            token_clusters: Clusters en índices de tokens
        """
        token_clusters = []
        
        for cluster in char_clusters:
            token_cluster = []
            for char_start, char_end in cluster:
                # Encontrar tokens que se superponen con este span
                span_tokens = []
                
                for token_idx, (token_start, token_end) in enumerate(offset_mapping):
                    # Ignorar tokens especiales ([CLS], [SEP], etc.)
                    if token_start == 0 and token_end == 0:
                        continue
                    
                    # Verificar superposición
                    overlap_start = max(token_start, char_start)
                    overlap_end = min(token_end, char_end)
                    
                    if overlap_start < overlap_end:  # Hay superposición
                        span_tokens.append(token_idx)
                
                if span_tokens:
                    # Tomar el primer y último token del span
                    token_start = min(span_tokens)
                    token_end = max(span_tokens)
                    token_cluster.append((token_start, token_end))
            
            if len(token_cluster) >= 2:  # Al menos 2 menciones para formar cluster
                token_clusters.append(token_cluster)
        
        return token_clusters
    
    def create_training_pairs(self, token_clusters: List[List[Tuple[int, int]]]):
        """
        Crea pares de entrenamiento positivo y negativo
        
        Args:
            token_clusters: Clusters en índices de tokens
        Returns:
            positive_pairs: Lista de pares (span_i, span_j) que son coreferentes
            negative_pairs: Lista de pares que no son coreferentes
        """
        positive_pairs = []
        negative_pairs = []
        
        # Extraer todos los spans únicos
        all_spans = []
        span_to_cluster = {}
        
        for cluster_id, cluster in enumerate(token_clusters):
            for span in cluster:
                all_spans.append(span)
                span_to_cluster[span] = cluster_id
        
        # Crear pares
        for i, span_i in enumerate(all_spans):
            for j, span_j in enumerate(all_spans):
                if i <= j:
                    continue
                
                if span_to_cluster[span_i] == span_to_cluster[span_j]:
                    positive_pairs.append((span_i, span_j))
                else:
                    negative_pairs.append((span_i, span_j))
        
        # Balancear pares positivos y negativos
        if len(negative_pairs) > len(positive_pairs) * 3:
            negative_pairs = random.sample(negative_pairs, len(positive_pairs) * 3)
        
        return positive_pairs, negative_pairs
    
    def __getitem__(self, idx):
        example = self.examples[idx]
        
        # Tokenizar
        encoding = self.tokenize_and_align(example.text)
        
        # Alinear clusters a tokens
        token_clusters = self.align_clusters_to_tokens(
            encoding['offset_mapping'],
            example.char_clusters
        )
        
        # Crear inputs para el modelo
        inputs = {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'text': example.text,
            'token_clusters': token_clusters,
            'original_clusters': example.char_clusters
        }
        
        # Para entrenamiento, crear etiquetas
        if self.is_training and token_clusters:
            positive_pairs, negative_pairs = self.create_training_pairs(token_clusters)
            
            # Combinar pares y crear etiquetas
            all_pairs = positive_pairs + negative_pairs
            labels = [1] * len(positive_pairs) + [0] * len(negative_pairs)
            
            # Mezclar
            combined = list(zip(all_pairs, labels))
            random.shuffle(combined)
            all_pairs, labels = zip(*combined) if combined else ([], [])
            
            inputs['span_pairs'] = all_pairs[:self.max_spans]
            inputs['labels'] = torch.tensor(labels[:self.max_spans], dtype=torch.float)
        
        return inputs

def collate_fn(batch):
    """Función para agrupar ejemplos en batch"""
    # Padding dinámico para input_ids y attention_mask
    max_len = max(len(item['input_ids']) for item in batch)
    
    input_ids = []
    attention_mask = []
    texts = []
    clusters = []
    
    for item in batch:
        pad_len = max_len - len(item['input_ids'])
        input_ids.append(F.pad(item['input_ids'], (0, pad_len)))
        attention_mask.append(F.pad(item['attention_mask'], (0, pad_len)))
        texts.append(item['text'])
        clusters.append(item.get('token_clusters', []))
    
    batch_dict = {
        'input_ids': torch.stack(input_ids),
        'attention_mask': torch.stack(attention_mask),
        'texts': texts,
        'clusters': clusters
    }
    
    # Si hay datos de entrenamiento
    if 'span_pairs' in batch[0]:
        span_pairs = [item['span_pairs'] for item in batch]
        labels = [item['labels'] for item in batch]
        
        # Encontrar máximo número de pares
        max_pairs = max(len(pairs) for pairs in span_pairs)
        
        # Padding para span_pairs y labels
        padded_span_pairs = []
        padded_labels = []
        
        for pairs, lab in zip(span_pairs, labels):
            pad_len = max_pairs - len(pairs)
            if pad_len > 0:
                # Padding con pares dummy y etiquetas -1
                pairs = pairs + [(-1, -1)] * pad_len
                lab = F.pad(lab, (0, pad_len), value=-1)
            padded_span_pairs.append(pairs)
            padded_labels.append(lab)
        
        batch_dict['span_pairs'] = padded_span_pairs
        batch_dict['labels'] = torch.stack(padded_labels)
    
    return batch_dict

## 5. Función de Pérdida y Métricas

In [ ]:
## 5. Función de Pérdida y Métricas

class CoreferenceLoss(nn.Module):
    """Pérdida para entrenamiento de coreferencia"""
    
    def __init__(self, margin: float = 1.0, dummy_weight: float = 0.1):
        super().__init__()
        self.margin = margin
        self.dummy_weight = dummy_weight
        self.bce_loss = nn.BCEWithLogitsLoss(reduction='none')
        
    def forward(self, 
                scores: List[torch.Tensor], 
                labels: torch.Tensor,
                span_pairs: List[List[Tuple[int, int]]]) -> torch.Tensor:
        """
        Args:
            scores: Lista de tensores de scores por batch
            labels: [batch_size, max_pairs] etiquetas
            span_pairs: Lista de pares de spans por batch
        Returns:
            loss: Tensor escalar
        """
        batch_losses = []
        
        for b in range(len(scores)):
            batch_scores = scores[b]
            batch_labels = labels[b]
            batch_pairs = span_pairs[b]
            
            if len(batch_scores) == 0 or len(batch_pairs) == 0:
                continue
            
            # Filtrar padding (-1 en labels)
            valid_mask = batch_labels != -1
            if not valid_mask.any():
                continue
            
            valid_scores = batch_scores[valid_mask]
            valid_labels = batch_labels[valid_mask]
            
            # Calcular pérdida binaria
            loss = self.bce_loss(valid_scores, valid_labels)
            
            # Pérdida para dummy antecedents
            dummy_mask = torch.tensor([pair[1] == -1 for pair in batch_pairs], 
                                     device=valid_scores.device)
            if dummy_mask.any():
                dummy_scores = valid_scores[dummy_mask]
                dummy_labels = valid_labels[dummy_mask]
                dummy_loss = self.bce_loss(dummy_scores, dummy_labels) * self.dummy_weight
                loss[dummy_mask] = loss[dummy_mask] + dummy_loss
            
            batch_losses.append(loss.mean())
        
        if not batch_losses:
            return torch.tensor(0.0, device=device, requires_grad=True)
        
        return torch.stack(batch_losses).mean()

def compute_coref_metrics(pred_clusters: List[List[Tuple[int, int]]],
                         gold_clusters: List[List[Tuple[int, int]]]) -> Dict[str, float]:
    """
    Calcula métricas de coreferencia (MUC, B³, CEAF simplificadas)
    
    Args:
        pred_clusters: Clusters predichos
        gold_clusters: Clusters de referencia
    Returns:
        metrics: Diccionario con métricas
    """
    if not pred_clusters and not gold_clusters:
        return {"precision": 0.0, "recall": 0.0, "f1": 0.0}
    
    # Convertir clusters a conjuntos de pares
    def clusters_to_pairs(clusters):
        pairs = set()
        for cluster in clusters:
            for i in range(len(cluster)):
                for j in range(i + 1, len(cluster)):
                    pairs.add((cluster[i], cluster[j]))
        return pairs
    
    pred_pairs = clusters_to_pairs(pred_clusters)
    gold_pairs = clusters_to_pairs(gold_clusters)
    
    # Calcular métricas básicas
    tp = len(pred_pairs & gold_pairs)
    fp = len(pred_pairs - gold_pairs)
    fn = len(gold_pairs - pred_pairs)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "tp": tp,
        "fp": fp,
        "fn": fn
    }

def evaluate_model_on_dataset(model: CoreferenceClusterModel,
                            dataset: CoreferenceDataset,
                            tokenizer: XLMRobertaTokenizer,
                            device: str = "cpu") -> Dict[str, float]:
    """
    Evalúa el modelo en un dataset completo
    
    Args:
        model: Modelo entrenado
        dataset: Dataset de evaluación
        tokenizer: Tokenizer
        device: Dispositivo
        
    Returns:
        Métricas de evaluación promediadas
    """
    model.eval()
    all_metrics = []
    
    with torch.no_grad():
        for i in tqdm(range(len(dataset)), desc="Evaluando"):
            # Obtener ejemplo del dataset
            example_data = dataset[i]
            text = example_data['text']
            
            # Predecir clusters
            result = predict_clusters(
                model=model,
                text=text,
                tokenizer=tokenizer,
                threshold=0.3,
                device=device
            )
            
            # Obtener clusters predichos
            pred_clusters = []
            if 'clusters' in result:
                for cluster in result['clusters']:
                    if isinstance(cluster[0], dict) and 'span' in cluster[0]:
                        # Formato con diccionarios
                        token_cluster = [mention['span'] for mention in cluster]
                        pred_clusters.append(token_cluster)
                    else:
                        # Formato directo
                        pred_clusters.append(cluster)
            
            # Obtener clusters reales
            gold_clusters = example_data.get('token_clusters', [])
            
            # Calcular métricas
            if pred_clusters or gold_clusters:
                metrics = compute_coref_metrics(pred_clusters, gold_clusters)
                all_metrics.append(metrics)
    
    # Calcular promedios
    if not all_metrics:
        return {
            'precision': 0.0,
            'recall': 0.0,
            'f1': 0.0,
            'examples': 0
        }
    
    avg_metrics = {
        'precision': np.mean([m['precision'] for m in all_metrics]),
        'recall': np.mean([m['recall'] for m in all_metrics]),
        'f1': np.mean([m['f1'] for m in all_metrics]),
        'tp': np.sum([m.get('tp', 0) for m in all_metrics]),
        'fp': np.sum([m.get('fp', 0) for m in all_metrics]),
        'fn': np.sum([m.get('fn', 0) for m in all_metrics]),
        'examples': len(all_metrics)
    }
    
    return avg_metrics

## 6. Funciones de Decodificación

In [ ]:
def decode_clusters_from_scores(scores: torch.Tensor,
                               span_indices: List[Tuple[int, int]],
                               threshold: float = 0.5) -> List[List[Tuple[int, int]]]:
    """
    Decodifica clusters a partir de scores de pares
    
    Args:
        scores: [num_spans, num_antecedents+1] scores para cada span
        span_indices: Lista de índices de spans
        threshold: Umbral para considerar coreferente
    Returns:
        clusters: Lista de clusters decodificados
    """
    if len(scores) == 0:
        return []
    
    num_spans = len(scores)
    
    # Encontrar el mejor antecedente para cada span
    best_antecedents = []
    for i in range(num_spans):
        # Ignorar dummy antecedent (último)
        span_scores = scores[i][:-1]
        
        if len(span_scores) == 0:
            best_antecedents.append(-1)
            continue
        
        # Aplicar sigmoid y encontrar máximo
        probs = torch.sigmoid(span_scores)
        max_prob, max_idx = torch.max(probs, dim=0)
        
        if max_prob > threshold:
            best_antecedents.append(max_idx.item())
        else:
            best_antecedents.append(-1)  # Ningún antecedente
    
    # Construir clusters usando union-find
    parent = list(range(num_spans))
    
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    
    def union(x, y):
        root_x = find(x)
        root_y = find(y)
        if root_x != root_y:
            parent[root_y] = root_x
    
    # Unir spans con sus antecedentes
    for i, antecedent in enumerate(best_antecedents):
        if antecedent != -1:
            union(i, antecedent)
    
    # Crear clusters
    clusters_dict = defaultdict(list)
    for i in range(num_spans):
        root = find(i)
        clusters_dict[root].append(span_indices[i])
    
    # Filtrar clusters con una sola mención
    clusters = [spans for spans in clusters_dict.values() if len(spans) > 1]
    
    return clusters

def predict_clusters(model: CoreferenceClusterModel,
                    text: str,
                    tokenizer: XLMRobertaTokenizer,
                    threshold: float = 0.5,
                    device: str = "cpu") -> Dict:
    """
    Predice clusters de coreferencia para un texto
    
    Args:
        model: Modelo entrenado
        text: Texto de entrada
        tokenizer: Tokenizer
        threshold: Umbral para coreferencia
        device: Dispositivo
    Returns:
        Dict con predicciones
    """
    model.eval()
    
    # Tokenizar
    encoding = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)
    
    with torch.no_grad():
        # Obtener scores y spans
        outputs = model(
            encoding["input_ids"],
            encoding["attention_mask"],
            return_spans=True
        )
    
    # Decodificar clusters para el primer batch (asumiendo batch_size=1)
    scores = outputs['scores'][0]
    span_indices = outputs['span_indices'][0]
    
    if len(scores) == 0:
        return {
            "text": text,
            "clusters": [],
            "spans": []
        }
    
    # Decodificar clusters
    clusters = decode_clusters_from_scores(scores, span_indices, threshold)
    
    # Convertir índices de tokens a texto
    text_clusters = []
    for cluster in clusters:
        text_cluster = []
        for start_idx, end_idx in cluster:
            # Obtener tokens
            token_ids = encoding["input_ids"][0][start_idx:end_idx+1]
            tokens = tokenizer.convert_ids_to_tokens(token_ids)
            
            # Convertir a texto (limpiando tokens especiales)
            span_text = tokenizer.convert_tokens_to_string(tokens)
            text_cluster.append({
                "span": (start_idx.item(), end_idx.item()),
                "text": span_text,
                "char_span": encoding.token_to_chars(0, start_idx).start,
                "char_end": encoding.token_to_chars(0, end_idx).end
            })
        text_clusters.append(text_cluster)
    
    return {
        "text": text,
        "clusters": text_clusters,
        "raw_clusters": clusters,
        "span_indices": span_indices
    }

In [ ]:
## 6b. Procesamiento de Textos Largos (Sliding Window) y Visualización

def sliding_window_coref(texto_largo, modelo, tokenizer, window_size=100, stride=50, threshold=0.3):
    """
    Procesa textos largos dividiéndolos en ventanas con solapamiento
    
    Args:
        texto_largo: Texto completo a procesar
        modelo: Modelo de coreferencia entrenado
        tokenizer: Tokenizer
        window_size: Tamaño de ventana en tokens
        stride: Paso de solapamiento en tokens
        threshold: Umbral para considerar coreferencia
    
    Returns:
        Dict con clusters unificados de todo el texto
    """
    # Tokenizar texto completo
    tokens = tokenizer.tokenize(texto_largo)
    num_tokens = len(tokens)
    
    print(f"📊 Texto largo: {num_tokens} tokens")
    print(f"🔲 Ventana: {window_size} tokens, Paso: {stride} tokens")
    
    # Dividir en ventanas
    ventanas = []
    inicio = 0
    while inicio < num_tokens:
        fin = min(inicio + window_size, num_tokens)
        ventana_tokens = tokens[inicio:fin]
        ventana_texto = tokenizer.convert_tokens_to_string(ventana_tokens)
        
        ventanas.append({
            'inicio': inicio,
            'fin': fin,
            'texto': ventana_texto,
            'tokens': ventana_tokens
        })
        
        if fin == num_tokens:
            break
        inicio += stride
    
    print(f"🪟 Procesando {len(ventanas)} ventanas...")
    
    # Procesar cada ventana
    todos_clusters = []
    for i, ventana in enumerate(ventanas):
        if i < 5:  # Mostrar progreso para primeras 5 ventanas
            print(f"  Ventana {i+1}/{len(ventanas)}: tokens {ventana['inicio']}-{ventana['fin']}")
        
        resultado = predict_clusters(
            model=modelo,
            text=ventana['texto'],
            tokenizer=tokenizer,
            threshold=threshold,
            device=device
        )
        
        # Ajustar índices al texto completo
        if 'raw_clusters' in resultado:
            for cluster in resultado['raw_clusters']:
                cluster_ajustado = []
                for start, end in cluster:
                    # Ajustar al texto completo
                    start_global = start + ventana['inicio']
                    end_global = end + ventana['inicio']
                    cluster_ajustado.append((start_global, end_global))
                todos_clusters.append(cluster_ajustado)
    
    # Unificar clusters entre ventanas
    clusters_finales = unificar_clusters_sliding(todos_clusters, tokens, tokenizer)
    
    # Convertir a formato final
    clusters_con_texto = []
    for cluster in clusters_finales:
        cluster_texto = []
        for start_token, end_token in cluster:
            tokens_span = tokens[start_token:end_token+1]
            texto_span = tokenizer.convert_tokens_to_string(tokens_span)
            cluster_texto.append({
                'text': texto_span,
                'token_span': (start_token, end_token),
                'texto_completo_pos': None
            })
        clusters_con_texto.append(cluster_texto)
    
    return {
        'text': texto_largo,
        'clusters': clusters_con_texto,
        'raw_clusters': clusters_finales,
        'num_ventanas': len(ventanas),
        'tokens_totales': num_tokens
    }


def unificar_clusters_sliding(lista_clusters, tokens, tokenizer, umbral_solapamiento=0.5):
    """
    Unifica clusters que se superponen entre ventanas diferentes
    
    Args:
        lista_clusters: Lista de clusters de todas las ventanas
        tokens: Lista de tokens del texto completo
        tokenizer: Tokenizer para reconstruir texto
        umbral_solapamiento: % mínimo de solapamiento para unir clusters
    
    Returns:
        Lista de clusters unificados
    """
    if not lista_clusters:
        return []
    
    # Convertir clusters a conjuntos de menciones únicas
    menciones_por_cluster = []
    for cluster in lista_clusters:
        menciones_set = set()
        for start, end in cluster:
            # Crear hash único para la mención
            texto = tokenizer.convert_tokens_to_string(tokens[start:end+1])
            menciones_set.add((start, end, texto))
        menciones_por_cluster.append(menciones_set)
    
    # Unificar clusters con menciones en común
    clusters_unificados = []
    usado = [False] * len(menciones_por_cluster)
    
    for i in range(len(menciones_por_cluster)):
        if usado[i]:
            continue
        
        cluster_actual = set(menciones_por_cluster[i])
        usado[i] = True
        
        # Buscar clusters similares
        for j in range(i+1, len(menciones_por_cluster)):
            if usado[j]:
                continue
            
            cluster_otro = menciones_por_cluster[j]
            
            # Calcular solapamiento
            interseccion = len(cluster_actual.intersection(cluster_otro))
            union = len(cluster_actual.union(cluster_otro))
            
            if union > 0 and interseccion / union >= umbral_solapamiento:
                cluster_actual = cluster_actual.union(cluster_otro)
                usado[j] = True
        
        # Convertir de vuelta a formato de índices
        cluster_final = []
        for start, end, texto in cluster_actual:
            cluster_final.append((start, end))
        
        # Ordenar por posición en el texto
        cluster_final.sort(key=lambda x: x[0])
        clusters_unificados.append(cluster_final)
    
    return clusters_unificados


def visualizar_clusters_sliding(texto, resultado, max_caracteres=1000):
    """
    Visualiza clusters con colores en el texto
    
    Args:
        texto: Texto original
        resultado: Resultado de sliding_window_coref o predict_clusters
        max_caracteres: Máximo de caracteres a mostrar
    """
    import random
    from IPython.display import HTML, display
    
    # Colores para HTML (para notebooks)
    colores_html = [
        '#FF6B6B', '#4ECDC4', '#FFD166', '#06D6A0', '#118AB2', '#EF476F',
        '#073B4C', '#7209B7', '#F72585', '#3A86FF', '#FB5607', '#8338EC'
    ]
    
    texto_truncado = texto[:max_caracteres] if len(texto) > max_caracteres else texto
    if len(texto) > max_caracteres:
        print(f"📏 Texto truncado a {max_caracteres} caracteres")
    
    print(f"\n📝 Texto analizado ({len(texto)} caracteres):")
    print("-" * 80)
    
    # Para visualización en notebooks, usamos HTML
    html_output = f"<div style='font-family: monospace; white-space: pre-wrap; background-color: #f5f5f5; padding: 15px; border-radius: 5px;'>"
    
    clusters = resultado.get('clusters', [])
    
    if not clusters:
        html_output += "No se encontraron clusters de coreferencia."
    else:
        # Crear marcadores
        marcadores = [None] * len(texto_truncado)
        
        for cluster_idx, cluster in enumerate(clusters):
            color = colores_html[cluster_idx % len(colores_html)]
            
            for mention in cluster:
                if 'char_span' in mention and 'char_end' in mention:
                    start = mention['char_span']
                    end = mention['char_end']
                    
                    if start < len(texto_truncado):
                        # Marcar la mención
                        for pos in range(start, min(end, len(texto_truncado))):
                            if marcadores[pos] is None:
                                marcadores[pos] = []
                            marcadores[pos].append((cluster_idx, color))
        
        # Construir texto HTML con colores
        i = 0
        while i < len(texto_truncado):
            char = texto_truncado[i]
            
            if marcadores[i] is not None and marcadores[i]:
                # Hay cluster(s) en esta posición
                clusters_here = marcadores[i]
                
                # Usar el primer cluster (podría haber superposición)
                cluster_idx, color = clusters_here[0]
                
                # Encontrar hasta dónde se extiende este marcador
                j = i
                while j < len(texto_truncado) and marcadores[j] is not None and any(cidx == cluster_idx for cidx, _ in marcadores[j]):
                    j += 1
                
                html_output += f"<span style='background-color: {color}; color: white; padding: 2px; border-radius: 3px;' title='Cluster {cluster_idx+1}'>"
                html_output += texto_truncado[i:j].replace('\n', '<br>')
                html_output += "</span>"
                i = j
            else:
                # Sin marcador
                html_output += char.replace('\n', '<br>')
                i += 1
    
    html_output += "</div>"
    
    # Mostrar HTML
    display(HTML(html_output))
    
    # Leyenda
    print("\n📌 Clusters identificados:")
    for cluster_idx, cluster in enumerate(clusters):
        color = colores_html[cluster_idx % len(colores_html)]
        menciones = [m['text'] for m in cluster[:3]]  # Mostrar solo primeras 3
        if len(cluster) > 3:
            menciones.append(f"... (+{len(cluster)-3} más)")
        print(f"  • Cluster {cluster_idx+1}: {menciones}")
    
    # Estadísticas
    print(f"\n📊 Resumen:")
    print(f"  • Clusters totales: {len(clusters)}")
    print(f"  • Menciones totales: {sum(len(c) for c in clusters)}")
    
    if 'num_ventanas' in resultado:
        print(f"  • Ventanas procesadas: {resultado['num_ventanas']}")
    
    return resultado


print("✅ Funciones de sliding window y visualización cargadas")

# Slidind para documentos largos

In [ ]:
## Prueba del Modelo con Sliding Window

# Primero verificar que el modelo está cargado
if 'model' not in globals() or 'tokenizer' not in globals():
    print("❌ Modelo no cargado. Ejecuta primero train_with_conllu_data()")
else:
    print("✅ Modelo cargado. Probando con texto de ejemplo...")
    
    # Texto de ejemplo para prueba
    texto_prueba = """El director general de la empresa anunció los resultados del trimestre. 
    El ejecutivo mostró cifras positivas. El directivo explicó que las ventas habían crecido 
    un 15% respecto al año anterior. Los analistas recibieron bien la noticia."""
    
    print(f"\n📝 Texto de prueba ({len(texto_prueba.split())} palabras):")
    print("-" * 80)
    print(f'"{texto_prueba[:100]}..."' if len(texto_prueba) > 100 else f'"{texto_prueba}"')
    
    # Probar con sliding window
    resultado = sliding_window_coref(
        texto_largo=texto_prueba,
        modelo=model,
        tokenizer=tokenizer,
        window_size=100,
        stride=50,
        threshold=0.3
    )
    
    print(f"\n📊 Resultados:")
    print(f"  • Clusters encontrados: {len(resultado.get('clusters', []))}")
    
    if 'num_ventanas' in resultado:
        print(f"  • Ventanas procesadas: {resultado['num_ventanas']}")
    
    # Mostrar clusters encontrados
    clusters = resultado.get('clusters', [])
    if clusters:
        print(f"\n🔍 Clusters identificados:")
        for i, cluster in enumerate(clusters[:3]):  # Mostrar solo primeros 3
            print(f"\n  Cluster {i+1} ({len(cluster)} menciones):")
            for j, mention in enumerate(cluster):
                print(f"    {j+1}. '{mention['text']}'")
        
        if len(clusters) > 3:
            print(f"\n  ... y {len(clusters) - 3} clusters más")
    else:
        print("\n⚠️  No se encontraron clusters")

## 7. Generación de Datos de Ejemplo

In [ ]:
def load_conllu_dataset(conllu_path: str, max_examples: int = None) -> List[CoreferenceExample]:
    """
    Carga dataset desde archivo CoNLL-U
    
    Args:
        conllu_path: Ruta al archivo .conllu
        max_examples: Límite de ejemplos (útil para pruebas)
    
    Returns:
        Lista de CoreferenceExample
    """
    print(f"📂 Cargando dataset CoNLL-U desde: {conllu_path}")
    
    # Usar el lector CoNLL-U
    examples = CoNLLUReader.load_from_conllu(conllu_path)
    
    if max_examples and len(examples) > max_examples:
        examples = examples[:max_examples]
        print(f"   (Limitado a {max_examples} ejemplos para pruebas)")
    
    # Estadísticas
    total_clusters = sum(len(ex.clusters) for ex in examples)
    total_mentions = sum(sum(len(cluster) for cluster in ex.clusters) for ex in examples)
    
    print(f"📊 Estadísticas del dataset:")
    print(f"   • Ejemplos cargados: {len(examples)}")
    print(f"   • Clusters totales: {total_clusters}")
    print(f"   • Menciones totales: {total_mentions}")
    print(f"   • Promedio menciones por cluster: {total_mentions/total_clusters:.2f}" 
          if total_clusters > 0 else "0")
    
    # Distribución de longitudes
    lengths = [len(ex.text.split()) for ex in examples]
    if lengths:
        print(f"📏 Distribución de longitudes:")
        print(f"   • Mínimo: {min(lengths)} palabras")
        print(f"   • Máximo: {max(lengths)} palabras")
        print(f"   • Promedio: {sum(lengths)/len(lengths):.1f} palabras")
        print(f"   • Mediana: {sorted(lengths)[len(lengths)//2]} palabras")
    
    return examples

def save_dataset_info(examples: List[CoreferenceExample], output_path: str):
    """
    Guarda información del dataset para referencia
    
    Args:
        examples: Lista de CoreferenceExample
        output_path: Ruta para guardar la información
    """
    dataset_info = {
        "num_examples": len(examples),
        "num_clusters": sum(len(ex.clusters) for ex in examples),
        "num_mentions": sum(sum(len(cluster) for cluster in ex.clusters) for ex in examples),
        "examples": []
    }
    
    for i, ex in enumerate(examples[:10]):  # Guardar primeros 10 como muestra
        dataset_info["examples"].append({
            "text_preview": ex.text[:100] + "..." if len(ex.text) > 100 else ex.text,
            "num_clusters": len(ex.clusters),
            "num_mentions": sum(len(cluster) for cluster in ex.clusters),
            "tokens": ex.tokens[:20] + ["..."] if len(ex.tokens) > 20 else ex.tokens
        })
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(dataset_info, f, ensure_ascii=False, indent=2)
    
    print(f"📝 Información del dataset guardada en: {output_path}")

# Ejemplo de uso:
# Cargar dataset CoNLL-U (descomentar para usar)
# conllu_file = "tu_dataset.conllu"
# examples = load_conllu_dataset(conllu_file, max_examples=1000)
# save_dataset_info(examples, "dataset_info.json")

## 8. Entrenamiento del Modelo

In [ ]:
## 8. Entrenamiento del Modelo

def train_model(model: CoreferenceClusterModel,
               train_dataset: CoreferenceDataset,
               val_dataset: CoreferenceDataset,
               batch_size: int = 4,
               epochs: int = 10,
               learning_rate: float = 2e-5,
               warmup_steps: int = 100):
    """Función principal de entrenamiento"""
    
    # Crear DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=0  # Cambiado a 0 para evitar problemas en notebooks
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=0  # Cambiado a 0 para evitar problemas en notebooks
    )
    
    # Configurar optimizador y scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    # Función de pérdida
    criterion = CoreferenceLoss()
    
    # Historial
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_f1': []
    }
    
    # Entrenamiento
    for epoch in range(epochs):
        print(f"\n{'='*60}")
        print(f"Época {epoch + 1}/{epochs}")
        print(f"{'='*60}")
        
        # Fase de entrenamiento
        model.train()
        train_loss = 0
        train_batches = 0
        
        progress_bar = tqdm(train_loader, desc="Entrenamiento")
        for batch in progress_bar:
            # Mover datos al dispositivo
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Forward pass
            scores = model(input_ids, attention_mask)
            
            # Calcular pérdida
            if 'labels' in batch and 'span_pairs' in batch:
                labels = batch['labels'].to(device)
                span_pairs = batch['span_pairs']
                loss = criterion(scores, labels, span_pairs)
            else:
                # Si no hay etiquetas, saltar
                continue
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
            train_loss += loss.item()
            train_batches += 1
            
            # Actualizar barra de progreso
            progress_bar.set_postfix({'loss': loss.item()})
        
        avg_train_loss = train_loss / train_batches if train_batches > 0 else 0
        history['train_loss'].append(avg_train_loss)
        
        # Fase de validación
        model.eval()
        val_loss = 0
        val_batches = 0
        all_metrics = []
        
        with torch.no_grad():
            val_progress = tqdm(val_loader, desc="Validación")
            for batch in val_progress:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                # Forward pass
                scores = model(input_ids, attention_mask, return_spans=True)
                
                # Calcular pérdida si hay etiquetas
                if 'labels' in batch and 'span_pairs' in batch:
                    labels = batch['labels'].to(device)
                    span_pairs = batch['span_pairs']
                    loss = criterion(scores['scores'], labels, span_pairs)
                    val_loss += loss.item()
                    val_batches += 1
                
                # Evaluar métricas para cada ejemplo
                for b in range(len(scores['scores'])):
                    if len(scores['scores'][b]) == 0:
                        continue
                    
                    # Decodificar clusters predichos
                    pred_clusters = decode_clusters_from_scores(
                        scores['scores'][b],
                        scores['span_indices'][b]
                    )
                    
                    # Obtener clusters reales
                    gold_clusters = batch['clusters'][b]
                    
                    # Calcular métricas
                    metrics = compute_coref_metrics(pred_clusters, gold_clusters)
                    all_metrics.append(metrics)
        
        avg_val_loss = val_loss / val_batches if val_batches > 0 else 0
        history['val_loss'].append(avg_val_loss)
        
        # Calcular métricas promedio
        if all_metrics:
            avg_precision = np.mean([m['precision'] for m in all_metrics])
            avg_recall = np.mean([m['recall'] for m in all_metrics])
            avg_f1 = np.mean([m['f1'] for m in all_metrics])
            history['val_f1'].append(avg_f1)
        else:
            avg_precision = avg_recall = avg_f1 = 0
        
        print(f"\nResumen Época {epoch + 1}:")
        print(f"  Pérdida Entrenamiento: {avg_train_loss:.4f}")
        print(f"  Pérdida Validación:    {avg_val_loss:.4f}")
        print(f"  Precisión Validación:  {avg_precision:.4f}")
        print(f"  Recall Validación:     {avg_recall:.4f}")
        print(f"  F1 Validación:         {avg_f1:.4f}")
        
        # Guardar checkpoint
        if (epoch + 1) % 5 == 0:
            checkpoint_path = f"checkpoint_epoch_{epoch+1}.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'val_f1': avg_f1
            }, checkpoint_path)
            print(f"  Checkpoint guardado: {checkpoint_path}")
    
    return history

## 9. Preparación de Datos y Entrenamiento

In [ ]:
# Modificar esta celda completamente

def prepare_and_train_with_conllu(conllu_path: str, 
                                 test_size: float = 0.2,
                                 max_examples: int = None,
                                 max_length: int = 256):
    """
    Prepara y entrena el modelo con datos CoNLL-U
    
    Args:
        conllu_path: Ruta al archivo .conllu
        test_size: Proporción para validación
        max_examples: Límite de ejemplos
        max_length: Longitud máxima de secuencia
    """
    print("="*80)
    print("PREPARACIÓN Y ENTRENAMIENTO CON DATOS CoNLL-U")
    print("="*80)
    
    # 1. Cargar dataset CoNLL-U
    print("\n1. 📂 Cargando dataset CoNLL-U...")
    examples = load_conllu_dataset(conllu_path, max_examples)
    
    if not examples:
        print("❌ Error: No se pudieron cargar ejemplos del archivo CoNLL-U")
        return
    
    # 2. Dividir en train/val
    print(f"\n2. 📊 Dividiendo datos ({int((1-test_size)*100)}% train, {int(test_size*100)}% validation)...")
    split_idx = int(len(examples) * (1 - test_size))
    train_examples = examples[:split_idx]
    val_examples = examples[split_idx:]
    
    print(f"   → Entrenamiento: {len(train_examples)} ejemplos")
    print(f"   → Validación: {len(val_examples)} ejemplos")
    
    # 3. Inicializar tokenizer y modelo
    print("\n3. 🤖 Inicializando modelo XLM-RoBERTa...")
    tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
    model = CoreferenceClusterModel("xlm-roberta-base").to(device)
    
    print(f"   → Modelo cargado: {sum(p.numel() for p in model.parameters()):,} parámetros")
    print(f"   → Tokenizer: {tokenizer.__class__.__name__}")
    
    # 4. Crear datasets
    print("\n4. 🛠️ Creando datasets de PyTorch...")
    train_dataset = CoreferenceDataset(
        examples=train_examples,
        tokenizer=tokenizer,
        max_length=max_length,
        max_spans=100,
        is_training=True
    )
    
    val_dataset = CoreferenceDataset(
        examples=val_examples,
        tokenizer=tokenizer,
        max_length=max_length,
        max_spans=100,
        is_training=False
    )
    
    # 5. Probar un batch
    print("\n5. 🔍 Probando batch de ejemplo...")
    try:
        sample_batch = next(iter(DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn)))
        print(f"   ✅ Batch creado exitosamente:")
        print(f"      • input_ids shape: {sample_batch['input_ids'].shape}")
        print(f"      • attention_mask shape: {sample_batch['attention_mask'].shape}")
        if 'labels' in sample_batch:
            print(f"      • labels shape: {sample_batch['labels'].shape}")
            print(f"      • span_pairs: {len(sample_batch['span_pairs'])} pares")
    except Exception as e:
        print(f"   ❌ Error al crear batch: {e}")
        return
    
    return model, tokenizer, train_dataset, val_dataset

# Ejecutar preparación (descomentar para usar)
# conllu_file = "tu_archivo.conllu"  # Cambia esto por tu archivo real
# model, tokenizer, train_dataset, val_dataset = prepare_and_train_with_conllu(
#     conllu_path=conllu_file,
#     test_size=0.2,
#     max_examples=1000,  # Limitar para pruebas
#     max_length=256
# )

## 10. Entrenamiento (Ejecutar esta celda para entrenar)

In [ ]:
## 10. Entrenamiento con Datos CoNLL-U

def train_with_conllu_data(conllu_file="tu_dataset.conllu", test_size=0.2, max_examples=None):
    """
    Función principal para entrenar con datos CoNLL-U reales
    """
    global model, tokenizer, train_dataset, val_dataset, history
    
    print("="*80)
    print("ENTRENAMIENTO CON DATOS CoNLL-U")
    print("="*80)
    
    # 1. Verificar que el archivo existe
    if not os.path.exists(conllu_file):
        print(f"❌ ERROR: No se encontró el archivo: {conllu_file}")
        print("Por favor, asegúrate de que el archivo CoNLL-U existe en la ruta especificada.")
        return None, None, None, None, None
    
    # 2. Cargar dataset CoNLL-U
    print(f"\n📂 Cargando dataset CoNLL-U desde: {conllu_file}")
    examples = load_conllu_dataset(conllu_file, max_examples)
    
    if not examples:
        print("❌ Error: No se pudieron cargar ejemplos del archivo CoNLL-U")
        return None, None, None, None, None
    
    # 3. Dividir en train/val
    print(f"\n📊 Dividiendo datos ({int((1-test_size)*100)}% train, {int(test_size*100)}% validation)...")
    split_idx = int(len(examples) * (1 - test_size))
    train_examples = examples[:split_idx]
    val_examples = examples[split_idx:]
    
    print(f"   → Entrenamiento: {len(train_examples)} ejemplos")
    print(f"   → Validación: {len(val_examples)} ejemplos")
    
    # 4. Inicializar tokenizer y modelo
    print("\n🤖 Inicializando modelo XLM-RoBERTa...")
    tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
    model = CoreferenceClusterModel("xlm-roberta-base").to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"   → Modelo cargado: {total_params:,} parámetros")
    
    # 5. Crear datasets
    print("\n🛠️ Creando datasets de PyTorch...")
    train_dataset = CoreferenceDataset(
        examples=train_examples,
        tokenizer=tokenizer,
        max_length=256,
        max_spans=100,
        is_training=True
    )
    
    val_dataset = CoreferenceDataset(
        examples=val_examples,
        tokenizer=tokenizer,
        max_length=256,
        max_spans=100,
        is_training=False
    )
    
    # 6. Probar un batch
    print("\n🔍 Probando batch de ejemplo...")
    try:
        sample_batch = next(iter(DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn)))
        print(f"   ✅ Batch creado exitosamente:")
        print(f"      • input_ids shape: {sample_batch['input_ids'].shape}")
        print(f"      • attention_mask shape: {sample_batch['attention_mask'].shape}")
        if 'labels' in sample_batch:
            print(f"      • labels shape: {sample_batch['labels'].shape}")
            print(f"      • span_pairs: {len(sample_batch['span_pairs'])} pares")
    except Exception as e:
        print(f"   ❌ Error al crear batch: {e}")
        return model, tokenizer, None, train_dataset, val_dataset
    
    # 7. Preguntar por entrenamiento
    print("\n" + "="*80)
    print("¿Quieres iniciar el entrenamiento ahora?")
    print("1. Sí, entrenar el modelo")
    print("2. No, solo preparar los datos")
    
    try:
        choice = input("\nElige una opción (1-2): ").strip()
    except:
        choice = "1"  # Por defecto en notebooks
    
    if choice == "2":
        print("\n✅ Datos preparados. Puedes entrenar más tarde ejecutando:")
        print("   history = train_model(model, train_dataset, val_dataset, batch_size=4, epochs=10)")
        history = None
        return model, tokenizer, history, train_dataset, val_dataset
    
    # 8. Parámetros de entrenamiento
    epochs = 10
    batch_size = 4
    learning_rate = 2e-5
    
    print(f"\n🚀 Iniciando entrenamiento...")
    print(f"   • Épocas: {epochs}")
    print(f"   • Batch size: {batch_size}")
    print(f"   • Learning rate: {learning_rate}")
    print(f"   • Dispositivo: {device}")
    
    # 9. Entrenar el modelo
    history = train_model(
        model=model,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        epochs=epochs,
        learning_rate=learning_rate,
        warmup_steps=50
    )
    
    # 10. Guardar modelo
    print("\n💾 Guardando modelo entrenado...")
    model_path = "modelo_coref_entrenado"
    os.makedirs(model_path, exist_ok=True)
    
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': {
            'model_name': 'xlm-roberta-base',
            'max_span_width': model.max_span_width,
            'max_num_spans': model.max_num_spans,
            'hidden_size': model.hidden_size
        },
        'training_info': {
            'epochs': epochs,
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'train_examples': len(train_examples),
            'val_examples': len(val_examples),
            'conllu_file': conllu_file
        }
    }, os.path.join(model_path, "model.pt"))
    
    tokenizer.save_pretrained(model_path)
    
    # Guardar historial
    with open(os.path.join(model_path, "training_history.json"), 'w') as f:
        json.dump({
            'train_loss': history['train_loss'],
            'val_loss': history['val_loss'],
            'val_f1': history['val_f1']
        }, f, indent=2)
    
    print(f"\n✅ Modelo guardado en: {model_path}/")
    
    return model, tokenizer, history, train_dataset, val_dataset

# ============================================================================
# EJECUCIÓN PRINCIPAL
# ============================================================================

print("\n" + "="*80)
print("INSTRUCCIONES PARA ENTRENAR EL MODELO")
print("="*80)
print("\nPara entrenar el modelo, necesitas un archivo CoNLL-U con anotaciones de coreferencia.")
print("\nPasos:")
print("1. Asegúrate de tener un archivo .conllu (ej: 'datos.conllu')")
print("2. Modifica la variable 'conllu_file' en la función train_with_conllu_data()")
print("3. Ejecuta la siguiente línea (descoméntala):")
print("\n   model, tokenizer, history, train_dataset, val_dataset = train_with_conllu_data()")

# Ejemplo de cómo ejecutar (descomentar):
# model, tokenizer, history, train_dataset, val_dataset = train_with_conllu_data(
#     conllu_file="tu_archivo.conllu",  # Cambia esto
#     test_size=0.2,
#     max_examples=1000  # Opcional: limita el número de ejemplos
# )

In [ ]:
## 10b. Inicialización Rápida para Pruebas (Sin Entrenamiento)

def inicializar_modelo_para_pruebas():
    """
    Inicializa un modelo básico sin necesidad de entrenar
    Útil para probar las funciones antes de entrenar con datos reales
    """
    global model, tokenizer
    
    print("🤖 Inicializando modelo XLM-RoBERTa para pruebas...")
    
    # Inicializar tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
    
    # Inicializar modelo con pesos aleatorios (no entrenado)
    model = CoreferenceClusterModel(
        model_name="xlm-roberta-base",
        max_span_width=10,
        max_num_spans=100
    ).to(device)
    
    print(f"✅ Modelo inicializado para pruebas:")
    print(f"   → Parámetros: {sum(p.numel() for p in model.parameters()):,}")
    print(f"   → Dispositivo: {device}")
    print(f"   → Tokenizer: {tokenizer.__class__.__name__}")
    
    # Crear datasets de ejemplo para estructura (sin datos reales)
    dummy_examples = [
        CoreferenceExample(
            text="Juan fue al mercado. Él compró manzanas.",
            tokens=["Juan", "fue", "al", "mercado", ".", "Él", "compró", "manzanas", "."],
            clusters=[[(0, 0), (5, 5)]],  # "Juan" y "Él"
            char_clusters=[[(0, 4), (25, 27)]]
        )
    ]
    
    dummy_dataset = CoreferenceDataset(
        examples=dummy_examples,
        tokenizer=tokenizer,
        max_length=128,
        max_spans=50,
        is_training=False
    )
    
    return model, tokenizer, dummy_dataset


# Ejecutar para inicializar (descomentar si quieres probar sin entrenar)
# model, tokenizer, dummy_dataset = inicializar_modelo_para_pruebas()
# print("\n✅ Modelo listo para pruebas básicas.")

## 11. Predicción y Evaluación

In [ ]:
## 11. Predicción y Evaluación - Versión Mejorada

def check_and_initialize_components():
    """
    Verifica y opcionalmente inicializa los componentes necesarios
    """
    global model, tokenizer
    
    print("🔍 Verificando componentes del sistema...")
    
    try:
        # Verificar si ya están definidos
        if 'model' not in globals() or 'tokenizer' not in globals():
            print("⚠️  Modelo no encontrado. Inicializando para pruebas...")
            inicializar_modelo_para_pruebas()
        else:
            print(f"✅ Modelo encontrado: {model.__class__.__name__}")
            print(f"✅ Tokenizer encontrado: {tokenizer.__class__.__name__}")
        
        # Verificar que el modelo esté en el dispositivo correcto
        model.to(device)
        print(f"✅ Modelo configurado en dispositivo: {device}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error al verificar componentes: {e}")
        print("\n💡 Solución: Ejecuta primero una de estas opciones:")
        print("   1. train_with_conllu_data() para entrenar con datos reales")
        print("   2. inicializar_modelo_para_pruebas() para pruebas rápidas")
        return False


def ejemplo_prediccion_rapida(texto_ejemplo=None):
    """
    Ejemplo rápido de predicción para probar el sistema
    """
    if texto_ejemplo is None:
        texto_ejemplo = """
        María González es la nueva gerente del departamento de tecnología. 
        La ingeniera tiene más de 10 años de experiencia en el sector. 
        Ella liderará un equipo de 20 desarrolladores. La Sra. González 
        anteriormente trabajó en Google y Microsoft.
        """
    
    print("🧪 Ejecutando ejemplo de predicción rápida...")
    print(f"📝 Texto de prueba:\n\"{texto_ejemplo[:100]}...\"")
    
    # Verificar/Inicializar componentes
    if not check_and_initialize_components():
        return None
    
    # Probar predicción directa
    print("\n🔮 Predicción básica:")
    resultado = predict_clusters(
        model=model,
        text=texto_ejemplo,
        tokenizer=tokenizer,
        threshold=0.3,
        device=device
    )
    
    # Mostrar resultados
    print(f"\n📊 Resultados:")
    print(f"  • Clusters encontrados: {len(resultado.get('clusters', []))}")
    
    if resultado.get('clusters'):
        for i, cluster in enumerate(resultado['clusters'][:3]):  # Mostrar primeros 3
            menciones = [m['text'] for m in cluster]
            print(f"  • Cluster {i+1}: {menciones}")
    else:
        print("  • No se encontraron clusters (esperado con modelo no entrenado)")
    
    return resultado


# Verificar componentes al cargar esta celda
if check_and_initialize_components():
    print("\n✅ Sistema listo para predicciones")
    print("\n💡 Prueba rápida (descomenta para ejecutar):")
    print("# resultado = ejemplo_prediccion_rapida()")
else:
    print("\n⚠️  Sistema no está completamente inicializado")

# Procesador inteligente para párrafos

In [ ]:
class ProcesadorPárrafos:
    """Procesador inteligente que decide automáticamente usar sliding window"""
    
    def __init__(self, modelo, tokenizador, max_tokens=450):
        self.modelo = modelo
        self.tokenizador = tokenizador
        self.max_tokens = max_tokens
    
    def procesar_texto(self, texto, umbral=0.3):
        """
        Procesa texto automáticamente, usando sliding window si es necesario
        """
        # 1. Calcular tokens
        tokens = self.tokenizador.tokenize(texto)
        num_tokens = len(tokens)
        
        print(f"📊 Análisis del texto:")
        print(f"   • Caracteres: {len(texto)}")
        print(f"   • Palabras: {len(texto.split())}")
        print(f"   • Tokens: {num_tokens}")
        print(f"   • Límite del modelo: {self.max_tokens} tokens")
        
        # 2. Decidir estrategia
        if num_tokens <= self.max_tokens:
            print(f"\n✅ Texto corto - Procesando directamente...")
            return predict_clusters(self.modelo, texto, self.tokenizador, umbral, device)
        else:
            print(f"\n⚠️  Texto largo - Usando Sliding Window...")
            
            # Calcular parámetros óptimos
            window_size = min(400, self.max_tokens - 50)  # Dejar margen
            stride = window_size // 2  # 50% de solapamiento
            
            print(f"   • Ventana: {window_size} tokens")
            print(f"   • Paso: {stride} tokens")
            print(f"   • Ventanas estimadas: {(num_tokens - window_size) // stride + 1}")
            
            return sliding_window_coref(
                texto_largo=texto,
                modelo=self.modelo,
                tokenizer=self.tokenizador,
                window_size=window_size,
                stride=stride,
                threshold=umbral
            )
    
    def procesar_multiples_parrafos(self, texto, separador='\n\n'):
        """
        Procesa múltiples párrafos por separado y luego unifica resultados
        """
        párrafos = [p.strip() for p in texto.split(separador) if p.strip()]
        print(f"📑 Procesando {len(párrafos)} párrafo(s)...")
        
        todos_resultados = []
        
        for idx, párrafo in enumerate(párrafos):
            print(f"\n   Párrafo {idx + 1}:")
            resultado = self.procesar_texto(párrafo)
            todos_resultados.append(resultado)
        
        # Unificar resultados entre párrafos
        return self.unificar_resultados_entre_parrafos(todos_resultados, texto)
    
    def unificar_resultados_entre_parrafos(self, resultados, texto_original):
        """
        Intenta conectar clusters entre diferentes párrafos
        """
        # Extraer todos los clusters
        todos_clusters = []
        for resultado in resultados:
            if 'raw_clusters' in resultado:
                todos_clusters.extend(resultado['raw_clusters'])
        
        # Unificar (simplificado - en realidad necesitarías lógica más compleja)
        clusters_unificados = unificar_clusters_sliding(
            todos_clusters,
            self.tokenizador.tokenize(texto_original),
            self.tokenizador
        )
        
        # Convertir a formato final
        clusters_con_texto = []
        for cluster in clusters_unificados:
            cluster_texto = []
            for start_token, end_token in cluster:
                tokens = self.tokenizador.tokenize(texto_original)
                tokens_span = tokens[start_token:end_token+1]
                texto_span = self.tokenizador.convert_tokens_to_string(tokens_span)
                cluster_texto.append({
                    'text': texto_span,
                    'token_span': (start_token, end_token)
                })
            clusters_con_texto.append(cluster_texto)
        
        return {
            'text': texto_original,
            'clusters': clusters_con_texto,
            'raw_clusters': clusters_unificados,
            'num_parrafos': len(resultados)
        }

# Inicializar procesador
procesador = ProcesadorPárrafos(model, tokenizer)

# Probar con diferentes tipos de texto
print("=" * 100)
print("PRUEBA DEL PROCESADOR INTELIGENTE")
print("=" * 100)

# Ejemplo 1: Texto corto
ejemplo_corto = "El presidente anunció medidas. El mandatario dijo que son urgentes."
print("\n1. 📝 TEXTO CORTO:")
resultado1 = procesador.procesar_texto(ejemplo_corto)
visualizar_clusters_sliding(ejemplo_corto, resultado1)

# Ejemplo 2: Texto largo (un párrafo)
print("\n\n2. 📄 TEXTO LARGO (1 párrafo):")
resultado2 = procesador.procesar_texto(documento_largo)
visualizar_clusters_sliding(documento_largo, resultado2)

# Ejemplo 3: Múltiples párrafos
texto_multi_parrafo = """
Primer párrafo: Carlos Méndez es el nuevo director. El ejecutivo tiene amplia experiencia.

Segundo párrafo: El Sr. Méndez anteriormente trabajó en grandes empresas. 
Allí, el profesional lideró equipos internacionales.

Tercer párrafo: En su nuevo puesto, Carlos implementará cambios. El director prometió mejoras.
"""

print("\n\n3. 📚 MÚLTIPLES PÁRRAFOS:")
resultado3 = procesador.procesar_multiples_parrafos(texto_multi_parrafo)
visualizar_clusters_sliding(texto_multi_parrafo, resultado3)

## 12. Evaluación Cuantitativa

In [ ]:
## 12. Evaluación en CoNLL-U Test

def evaluate_on_conllu_test(test_conllu_path: str):
    """
    Evalúa el modelo en un conjunto de test CoNLL-U
    """
    # Verificar componentes
    if 'model' not in globals() or 'tokenizer' not in globals():
        print("❌ Modelo no está inicializado. Ejecuta primero train_with_conllu_data()")
        return None
    
    print(f"\n{'='*80}")
    print(f"EVALUACIÓN EN CONJUNTO DE TEST CoNLL-U")
    print(f"{'='*80}")
    
    # 1. Cargar datos de test
    print("\n1. 📂 Cargando datos de test...")
    test_examples = load_conllu_dataset(test_conllu_path)
    
    if not test_examples:
        print("❌ Error: No se pudieron cargar ejemplos de test")
        return None
    
    # 2. Crear dataset de test
    test_dataset = CoreferenceDataset(
        examples=test_examples,
        tokenizer=tokenizer,
        max_length=256,
        max_spans=100,
        is_training=False
    )
    
    # 3. Evaluar
    print("\n2. 📊 Evaluando modelo...")
    test_metrics = evaluate_model_on_dataset(
        model=model,
        dataset=test_dataset,
        tokenizer=tokenizer,
        device=device
    )
    
    # 4. Mostrar resultados
    print(f"\n3. 📈 Resultados en Test CoNLL-U:")
    print(f"   • Ejemplos evaluados: {test_metrics['examples']}")
    print(f"   • Precisión: {test_metrics['precision']:.4f}")
    print(f"   • Recall:    {test_metrics['recall']:.4f}")
    print(f"   • F1 Score:  {test_metrics['f1']:.4f}")
    
    return test_metrics

# Para usar:
# test_metrics = evaluate_on_conllu_test("ruta/test.conllu")

In [ ]:
def evaluate_on_conllu_test(model: CoreferenceClusterModel,
                          tokenizer: XLMRobertaTokenizer,
                          test_conllu_path: str,
                          device: str = "cpu"):
    """
    Evalúa el modelo en un conjunto de test CoNLL-U
    
    Args:
        model: Modelo entrenado
        tokenizer: Tokenizer
        test_conllu_path: Ruta al archivo .conllu de test
        device: Dispositivo
    
    Returns:
        Métricas de evaluación
    """
    print(f"\n{'='*80}")
    print(f"EVALUACIÓN EN CONJUNTO DE TEST CoNLL-U")
    print(f"{'='*80}")
    
    # 1. Cargar datos de test
    print("\n1. 📂 Cargando datos de test...")
    test_examples = load_conllu_dataset(test_conllu_path)
    
    if not test_examples:
        print("❌ Error: No se pudieron cargar ejemplos de test")
        return None
    
    # 2. Crear dataset de test
    test_dataset = CoreferenceDataset(
        examples=test_examples,
        tokenizer=tokenizer,
        max_length=256,
        max_spans=100,
        is_training=False
    )
    
    # 3. Evaluar
    print("\n2. 📊 Evaluando modelo...")
    test_metrics = evaluate_model_on_dataset(
        model=model,
        dataset=test_dataset,
        tokenizer=tokenizer,
        device=device
    )
    
    # 4. Mostrar resultados
    print(f"\n3. 📈 Resultados en Test CoNLL-U:")
    print(f"   • Ejemplos evaluados: {test_metrics['examples']}")
    print(f"   • Precisión: {test_metrics['precision']:.4f}")
    print(f"   • Recall:    {test_metrics['recall']:.4f}")
    print(f"   • F1 Score:  {test_metrics['f1']:.4f}")
    
    # 5. Ejemplo de predicción
    print(f"\n4. 🔍 Ejemplo de predicción:")
    if test_examples:
        test_example = test_examples[0]
        print(f"   Texto: \"{test_example.text[:100]}...\"")
        
        result = predict_clusters(
            model=model,
            text=test_example.text,
            tokenizer=tokenizer,
            threshold=0.3,
            device=device
        )
        
        print(f"   Clusters predichos: {len(result.get('clusters', []))}")
        print(f"   Clusters reales: {len(test_example.clusters)}")
        
        # Comparar
        print(f"\n   Comparación (primeros 2 clusters):")
        for i in range(min(2, len(result.get('clusters', [])))):
            if i < len(result['clusters']):
                pred_texts = [m['text'] for m in result['clusters'][i]]
                print(f"   Predicción {i+1}: {pred_texts}")
            if i < len(test_example.clusters):
                # Convertir índices a texto
                cluster_texts = []
                for start, end in test_example.clusters[i]:
                    if start < len(test_example.tokens) and end < len(test_example.tokens):
                        cluster_texts.append(' '.join(test_example.tokens[start:end+1]))
                print(f"   Real {i+1}:      {cluster_texts}")
    
    return test_metrics

# Ejemplo de uso (descomentar):
# test_metrics = evaluate_on_conllu_test(
#     model=model,
#     tokenizer=tokenizer,
#     test_conllu_path="ruta/test.conllu",
#     device=device
# )

## 13. Visualización de Embeddings

In [ ]:
## 13. Visualización de Embeddings

def visualize_span_embeddings(text: str):
    """Visualiza embeddings de spans usando PCA"""
    
    try:
        from sklearn.decomposition import PCA
        import matplotlib.pyplot as plt
    except ImportError:
        print("Instala scikit-learn y matplotlib para visualización")
        return
    
    # Verificar componentes
    if 'model' not in globals() or 'tokenizer' not in globals():
        print("❌ Modelo no está inicializado")
        return
    
    # Obtener embeddings
    model.eval()
    with torch.no_grad():
        encoding = tokenizer(text, return_tensors="pt").to(device)
        outputs = model(encoding["input_ids"], encoding["attention_mask"], return_spans=True)
    
    if len(outputs['span_embeddings']) == 0 or len(outputs['span_embeddings'][0]) == 0:
        print("No se encontraron spans")
        return
    
    span_embeddings = outputs['span_embeddings'][0].cpu().numpy()
    span_indices = outputs['span_indices'][0]
    
    if len(span_embeddings) == 0:
        print("No se encontraron spans")
        return
    
    # Reducir dimensionalidad
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(span_embeddings)
    
    # Obtener texto de cada span
    span_texts = []
    for start, end in span_indices:
        tokens = tokenizer.convert_ids_to_tokens(
            encoding["input_ids"][0][start:end+1]
        )
        span_text = tokenizer.convert_tokens_to_string(tokens)
        span_texts.append(span_text[:20])  # Limitar longitud
    
    # Visualizar
    plt.figure(figsize=(12, 10))
    
    # Crear scatter plot
    scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                         alpha=0.6, s=100)
    
    # Añadir etiquetas
    for i, (x, y) in enumerate(embeddings_2d):
        plt.annotate(f"{i}: {span_texts[i]}", 
                    (x, y), 
                    fontsize=8,
                    alpha=0.7)
    
    plt.title("Embeddings de Spans (PCA)")
    plt.xlabel("Componente Principal 1")
    plt.ylabel("Componente Principal 2")
    plt.grid(True, alpha=0.3)
    
    # Añadir tabla de spans
    print("\nSpans encontrados:")
    for i, (span, text) in enumerate(zip(span_indices, span_texts)):
        print(f"  {i}: [{span[0]}-{span[1]}] '{text}'")
    
    plt.show()

# Ejemplo de uso:
# visualize_span_embeddings("Juan fue al mercado. Él compró manzanas.")

In [ ]:
def load_trained_model(model_path: str = None):
    """
    Carga un modelo entrenado o usa el modelo actual
    """
    if model_path and os.path.exists(model_path):
        # Cargar modelo guardado
        checkpoint = torch.load(f"{model_path}/model.pt", map_location=device)
        model = CoreferenceClusterModel(**checkpoint['model_config'])
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)
        print(f"Modelo cargado desde {model_path}")
    else:
        # Usar el modelo actual del notebook
        print("Usando modelo actual del notebook")
        model = model  # ya está definido en el notebook
        tokenizer = tokenizer  # ya está definido en el notebook
    
    return model, tokenizer

def analizar_coreferencias(texto, modelo=None, tokenizer=None, usar_sliding=True):
    """
    Función principal simplificada para analizar coreferencias
    
    Args:
        texto: Texto a analizar
        usar_sliding: True para usar sliding window automáticamente
    """
    if modelo is None or tokenizer is None:
        # Cargar modelo por defecto
        modelo, tokenizer = load_trained_model()
    
    print("🧠 Analizando coreferencias...")
    print(f"Texto de entrada ({len(texto)} caracteres)")
    print("-" * 80)
    
    # Decidir estrategia basada en longitud
    tokens = tokenizer.tokenize(texto)
    
    if len(tokens) <= 450 or not usar_sliding:
        print("✅ Usando procesamiento directo")
        resultado = predict_clusters(modelo, texto, tokenizer, threshold=0.3, device=device)
    else:
        print(f"⚠️  Texto largo ({len(tokens)} tokens) - Usando Sliding Window")
        # Parámetros óptimos para español
        window_size = 400
        stride = 200
        
        resultado = sliding_window_coref(
            texto_largo=texto,
            modelo=modelo,
            tokenizer=tokenizer,
            window_size=window_size,
            stride=stride,
            threshold=0.3
        )
    
    # Mostrar resultados
    print(f"\n📊 Resultados:")
    print(f"   • Clusters encontrados: {len(resultado.get('clusters', []))}")
    
    if 'num_ventanas' in resultado:
        print(f"   • Ventanas procesadas: {resultado['num_ventanas']}")
    
    print("\n📌 Clusters identificados:")
    for i, cluster in enumerate(resultado.get('clusters', [])):
        print(f"\n   Cluster {i+1} ({len(cluster)} menciones):")
        for j, mention in enumerate(cluster):
            print(f"     {j+1}. \"{mention['text']}\"")
    
    return resultado

# Probar con un ejemplo
print("=" * 100)
print("PRUEBA DE LA FUNCIÓN PRINCIPAL")
print("=" * 100)

mi_texto = """
El equipo de desarrollo presentó el nuevo software. Los programadores trabajaron durante meses.
Los ingenieros estaban satisfechos con el resultado. El producto fue bien recibido por los usuarios.
Los desarrolladores ya planean la siguiente versión.
"""

resultado_final = analizar_coreferencias(mi_texto, model, tokenizer, usar_sliding=True)

## 14. Exportación del Modelo

In [ ]:
def export_model_for_production(model: CoreferenceClusterModel,
                               tokenizer: XLMRobertaTokenizer,
                               export_path: str = "coreference_model_export"):
    """Exporta el modelo para producción"""
    
    # Guardar modelo completo
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': {
            'model_name': 'xlm-roberta-base',
            'max_span_width': model.max_span_width,
            'max_num_spans': model.max_num_spans,
            'hidden_size': model.hidden_size
        }
    }, f"{export_path}/model.pt")
    
    # Guardar tokenizer
    tokenizer.save_pretrained(export_path)
    
    # Crear script de inferencia
    inference_script = """
import torch
import json
from transformers import XLMRobertaTokenizer
from coreference_model import CoreferenceClusterModel

class CoreferencePredictor:
    def __init__(self, model_path: str):
        # Cargar configuración
        checkpoint = torch.load(f"{model_path}/model.pt", map_location='cpu')
        
        # Inicializar modelo
        self.model = CoreferenceClusterModel(**checkpoint['model_config'])
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        # Cargar tokenizer
        self.tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)
        
    def predict(self, text: str, threshold: float = 0.3):
        # Tokenizar
        encoding = self.tokenizer(text, return_tensors="pt")
        
        # Predecir
        with torch.no_grad():
            outputs = self.model(**encoding, return_spans=True)
        
        # Procesar resultados
        clusters = []
        for i in range(len(outputs['scores'])):
            # Decodificar clusters
            # ... (código de decodificación)
            pass
        
        return clusters
"""
    
    with open(f"{export_path}/inference.py", "w") as f:
        f.write(inference_script)
    
    print(f"Modelo exportado a {export_path}")
    print("Archivos creados:")
    print(f"  - {export_path}/model.pt (modelo PyTorch)")
    print(f"  - {export_path}/inference.py (script de inferencia)")
    print(f"  - {export_path}/tokenizer.json (configuración del tokenizer)")

# Exportar modelo
# export_model_for_production(model, tokenizer, "coreference_model")

## 16. Limitaciones y Mejoras Futuras

In [ ]:
print("""
LIMITACIONES ACTUALES Y MEJORAS FUTURAS:

1. Limitaciones:
   - Modelo entrenado con datos sintéticos limitados
   - Máxima longitud de texto: 512 tokens
   - No considera features lingüísticas complejas (género, número, etc.)
   - Requiere umbral manual para decodificación

2. Mejoras posibles:
   - Entrenar con datasets reales (Ontonotes, CorefUD)
   - Implementar beam search para decodificación
   - Añadir features lingüísticas (POS tags, dependency parsing)
   - Implementar modelos más avanzados (SpanBERT, CorefQA)
   - Añadir soporte para documentos largos (chunking)

3. Para producción:
   - Optimizar para inferencia rápida
   - Añadir caché de embeddings
   - Implementar batch processing eficiente
   - Crear API REST
""")

## 17. Guardar Notebook Completado

In [ ]:
# Guardar una copia del notebook
from IPython.display import HTML

download_script = """
<script>
function downloadNotebook() {
    var notebook = IPython.notebook;
    var notebook_name = notebook.notebook_name;
    var notebook_path = notebook.notebook_path;
    
    // Crear enlace de descarga
    var link = document.createElement('a');
    link.href = notebook_path;
    link.download = notebook_name;
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
}
</script>

<button onclick="downloadNotebook()" style="padding: 10px 20px; background-color: #4CAF50; color: white; border: none; border-radius: 5px; cursor: pointer;">
    Descargar Notebook
</button>
"""

display(HTML(download_script))